In [ ]:
# Imports
import rawpy
import numpy as np
import sys
import argparse
from PIL import Image
from scipy import interpolate

# Programming Assignment 3
## Due Date: 23:59, April 9th
## Name: YOUR NAME HERE

In this programming assignment you will complete an image signal processing pipeline.

## Instructions
1. Please write your name above in the Name field below the due date.
2. Please modify the `demosaic` function. A correct demosaicing function implementation is worth **10 points**.
3. You MAY NOT use any external libraries to implement demosaicing or tone mapping.
4. Do not touch any other portions of the code!

## What and how to submit
Please run the **entirety of the Jupyter Notebook**.
Even if you haven't implemented the `tone_mapping` function, the Jupyter Notebook should still generate images.
Submit _just_ the Jupyter Notebook to Canvas by the due date.
Please _do not_ submit any other files.

## Pipeline Code

In [ ]:
def read_image(input_image_path: str) -> rawpy.RawPy:
    """
    Reads a raw image at a specified path
    """
    return rawpy.imread(input_image_path)

In [ ]:
def extract_metadata(raw: rawpy.RawPy) -> np.ndarray:
    # raw.raw_colors raw_colors is a numerical mask; we instead generate a char mask so that we can check R/G/B by names

    # This is based on spatial raster order on the CFA; 01/32 on iPhone; 23/10 on Pixel
    cfa_pattern_id = np.array(raw.raw_pattern)

    # This is baesd on numerical order above; RGBG on iPhone; RGBG on Pixel; 
    color_desc = np.frombuffer(raw.color_desc, dtype=np.byte)

    # This tile is based on the raster order
    # https://stackoverflow.com/questions/14639496/how-to-create-a-numpy-array-of-arbitrary-length-strings
    tile_pattern = np.array([[chr(color_desc[cfa_pattern_id[0, 0]]), chr(color_desc[cfa_pattern_id[0, 1]])],
                             [chr(color_desc[cfa_pattern_id[1, 0]]), chr(color_desc[cfa_pattern_id[1, 1]])]], dtype=object)
    cfa_pattern_rgb = np.array(tile_pattern, copy=True) # make a deep copy

    return cfa_pattern_rgb

In [ ]:
def subtract_bl_norm(raw: rawpy.RawPy) -> np.ndarray:
    """
    Subtract black level and normalize
    """
    black = np.reshape(raw.black_level_per_channel, (2, 2))
    black = np.tile(black, (raw.raw_image.shape[0]//2, raw.raw_image.shape[1]//2))
    return (raw.raw_image - black) / (raw.white_level - black)

In [ ]:
def demosaic(greyscale_img: np.ndarray, cfa_pattern_rgb: np.ndarray) -> np.ndarray:
    
    ### Implement your demosaicing algorithm here! ###
    
    ##################################################

    return demosaic_img

In [ ]:
def apply_wb_cc(raw: rawpy.RawPy, demosaic_img: np.ndarray) -> np.ndarray:
    """
    Apply white balance and color correction
    """
    
    # https://www.pythoninformer.com/python-libraries/numpy/index-and-slice/
    # form a Nx3 array from the image pixels
    flat_img = np.stack((demosaic_img[:,:,0].flatten(),
                         demosaic_img[:,:,1].flatten(),
                         demosaic_img[:,:,2].flatten()))
    
    # White Balance
    wb_mat = np.array([[raw.camera_whitebalance[0], 0, 0],
                       [0, raw.camera_whitebalance[1], 0],
                       [0, 0, raw.camera_whitebalance[2]]])
    flat_img = np.matmul(wb_mat, flat_img)
    
    # Color Correction
    cc_mat = raw.color_matrix[0:3, 0:3]
    
    flat_img = np.clip(np.matmul(cc_mat, flat_img), 0, 1)

    
    color_img = np.stack((flat_img[0].reshape(raw.raw_image.shape[0], raw.raw_image.shape[1]),
                               flat_img[1].reshape(raw.raw_image.shape[0], raw.raw_image.shape[1]),
                               flat_img[2].reshape(raw.raw_image.shape[0], raw.raw_image.shape[1])), axis=2)
    return color_img

In [ ]:
def tone_mapping(color_img: np.ndarray) -> np.ndarray:
    
    '''
    Apply tone mapping
    '''
    # Reinhard E, Stark M, Shirley P, et al. Photographic tone reproduction for digital images[M]//Seminal Graphics Papers: Pushing the Boundaries, Volume 2. 2023: 661-670.
    # https://dl.acm.org/doi/pdf/10.1145/566654.566575

    Lw = 0.22

    return np.clip(color_img * (1 + color_img / (Lw ** 2)) / (1 + color_img), 0, 1)

In [ ]:
def apply_gamma(color_img: np.ndarray) -> np.ndarray:
    """
    Apply gamma curve
    """
    
    i = color_img < 0.0031308
    j = np.logical_not(i)
    color_img[i] = 323 / 25 * color_img[i]
    color_img[j] = 211 / 200 * color_img[j] ** (5 / 12) - 11 / 200
    
    return color_img

In [ ]:
def display_image(color_img: np.ndarray):
    """
    Displays final image on Jupyter notebook
    """
    display(Image.fromarray((color_img * 256).astype(np.uint8), 'RGB'))

In [ ]:
def pipeline(input_image_path: str):
    """
    Run an image through the entire pipeline
    """
    raw = read_image(input_image_path)
    cfa_pattern_rgb = extract_metadata(raw)
    greyscale_img = subtract_bl_norm(raw)
    demosaic_img = demosaic(greyscale_img, cfa_pattern_rgb)
    color_img = apply_wb_cc(raw, demosaic_img)
    color_img = tone_mapping(color_img)
    color_img = apply_gamma(color_img)
    display_image(color_img)

## Display Code

In [ ]:
pipeline('person.dng')

In [ ]:
pipeline('flower.dng')

In [ ]:
pipeline('tree.dng')